In [1]:
import os
import sys
from neo4j import GraphDatabase
from pymilvus import utility, connections
from minio import Minio
from minio.error import S3Error
from dotenv import dotenv_values

# --- 强行加载最新的 .env 配置 ---
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
env_path = os.path.join(project_root, '.env')
config = dotenv_values(env_path)

if not config:
    print(f"❌ 错误：无法从 {env_path} 加载配置。请检查 .env 文件是否存在且格式正确。")
else:
    # --- 从加载的配置中手动获取值 ---
    NEO4J_URI = config.get("NEO4J_URI")
    NEO4J_USER = config.get("NEO4J_USER")
    NEO4J_PASSWORD = config.get("NEO4J_PASSWORD")
    
    MINIO_ENDPOINT = config.get("MINIO_ENDPOINT")
    MINIO_ACCESS_KEY = config.get("MINIO_ACCESS_KEY")
    MINIO_SECRET_KEY = config.get("MINIO_SECRET_KEY")
    # 手动从 config.yaml 模拟获取 bucket name
    MINIO_BUCKET_NAME = "documents" 
    
    MILVUS_URI = config.get("MILVUS_URI")
    # 手动从 config.yaml 模拟获取 collection name
    MILVUS_COLLECTION_NAME = "bridgerag_collection"

    print("--- 配置强行加载成功 ---")
    print(f"Neo4j URI: {NEO4J_URI}")
    print(f"MinIO Endpoint: {MINIO_ENDPOINT}")
    print(f"Milvus URI: {MILVUS_URI}")
    print("---------------------\n")

    # --- 1. 测试 Neo4j 连接 ---
    def test_neo4j_connection():
        print("--- 1. 正在测试 Neo4j 连接 ---")
        try:
            driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
            with driver.session() as session:
                result = session.run("RETURN 'Connection to Neo4j successful!' AS message")
                message = result.single()["message"]
                print(f"✅ Neo4j: {message}")
            driver.close()
        except Exception as e:
            print(f"❌ Neo4j 连接失败: {e}")

    # --- 2. 测试 MinIO 连接 ---
    def test_minio_connection():
        print("\n--- 2. 正在测试 MinIO 连接 ---")
        try:
            # MinIO 的 endpoint 需要移除协议前缀
            endpoint_clean = MINIO_ENDPOINT.replace('http://', '').replace('https://', '')
            client = Minio(
                endpoint_clean,
                access_key=MINIO_ACCESS_KEY,
                secret_key=MINIO_SECRET_KEY,
                secure=False
            )
            found = client.bucket_exists(MINIO_BUCKET_NAME)
            if not found:
                client.make_bucket(MINIO_BUCKET_NAME)
                print(f"Bucket '{MINIO_BUCKET_NAME}' 不存在，已为您创建。")
            else:
                print(f"Bucket '{MINIO_BUCKET_NAME}' 已存在。")
            print(f"✅ MinIO: 连接成功。")
        except Exception as e:
            print(f"❌ MinIO 连接或操作失败: {e}")

    # --- 3. 测试 Milvus 连接 ---
    def test_milvus_connection():
        print("\n--- 3. 正在测试 Milvus 连接 ---")
        try:
            uri_parts = MILVUS_URI.replace("http://", "").split(':')
            host = uri_parts[0]
            port = uri_parts[1]
            
            connections.connect("default", host=host, port=port)
            has_collection = utility.has_collection(MILVUS_COLLECTION_NAME)
            if not has_collection:
                print(f"集合 '{MILVUS_COLLECTION_NAME}' 不存在。")
            else:
                print(f"集合 '{MILVUS_COLLECTION_NAME}' 已存在。")
            print(f"✅ Milvus: 连接成功。")
            connections.disconnect("default")
        except Exception as e:
            print(f"❌ Milvus 连接失败: {e}")

    # --- 执行所有测试 ---
    test_neo4j_connection()
    test_minio_connection()
    test_milvus_connection()

--- 配置强行加载成功 ---
Neo4j URI: bolt://localhost:7687
MinIO Endpoint: http://localhost:9000
Milvus URI: http://localhost:19530
---------------------

--- 1. 正在测试 Neo4j 连接 ---
✅ Neo4j: Connection to Neo4j successful!

--- 2. 正在测试 MinIO 连接 ---
Bucket 'documents' 已存在。
✅ MinIO: 连接成功。

--- 3. 正在测试 Milvus 连接 ---
集合 'bridgerag_collection' 不存在。
✅ Milvus: 连接成功。
